<a href="https://colab.research.google.com/github/Serge3leo/temp-cola/blob/main/stackoverflow-8585880.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Best way to handle multiple exit points in Fortran
# https://stackoverflow.com/a/79245899/8585880
# Install required packages
!apt-get update -qq
!apt-get install -qq gfortran
!pip install -qq fortran-magic
# Workaround colab bug
!pip install -qq meson charset-normalizer ninja cmake pkgconfig

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [2]:
%load_ext fortranmagic
# Workaround colab bug
%fortran_config --backend meson -v

New default arguments for %fortran:
	--backend meson -v


In [3]:
!echo 1 2 3 4 > 1234.txt
!echo 1.0 > 1_0.txt
filenames = ["1234.txt", "1_0.txt", "no-file"]

In [4]:
%%fortran -v
subroutine read_params(filename, params, ios)
   implicit none
   ! Argument Declarations !
   character(len=*), intent(in) :: filename
   integer, dimension(4), intent(out) :: params
   integer, intent(out) :: ios
   ! Variable Declarations
   integer :: i, iounit

   iounit = 1
   open(unit=iounit, status="old", file=filename, &
        err=10, iostat=ios)
   read(iounit, *, err=10, iostat=ios) (params(i), i=1, 4)
10 close(iounit)

end subroutine read_params


Ok. The following fortran objects are ready to use: read_params


In [5]:
for f in filenames:
    params, ios = read_params(f)
    if 0 == ios:
        print(f"params: {params} f:{f}")
    else:
        print(f"ios:{ios} f:{f}")

params: [1 2 3 4] f:1234.txt
ios:5010 f:1_0.txt
ios:2 f:no-file


In [6]:
%%fortran -v
subroutine read_params_msg(filename, params, ios, iom)
   implicit none
   ! Argument Declarations !
   character(len=*), intent(in) :: filename
   integer, dimension(4), intent(out) :: params
   integer, intent(out) :: ios
   character(len=256), intent(out) :: iom
   ! Variable Declarations
   integer :: i, iounit

   iounit = 1
   iom = ""
   open(unit=iounit, status="old", file=filename, &
        err=10, iostat=ios, iomsg=iom)
   read(iounit, *, err=10, iostat=ios, iomsg=iom) (params(i), i=1, 4)
10 close(iounit)

end subroutine read_params_msg


Ok. The following fortran objects are ready to use: read_params_msg


In [7]:
for f in filenames:
    params, ios, iom = read_params_msg(f)
    if 0 == ios:
        print(f"params: {params} f:{f}")
    else:
        print(f"ios:{ios} iom:{iom} f:{f}")

params: [1 2 3 4] f:1234.txt
ios:5010 iom:b'Bad integer for item 1 in list input' f:1_0.txt
ios:2 iom:b"Cannot open file 'no-file': No such file or directory" f:no-file


### **Warning. Next is the option of calling an exception in the callback, but, alas, this option is unstable.**
For some versions of python, and maybe for some OS, an exception in the callback causes python to crash.

In [8]:
def fmf2py_module(fn: str) -> "module":
    import sys

    fn_modules = [m for m in sys.modules.values()
                  if fn in m.__dict__
                  if globals()[fn] == m.__dict__[fn]
                  if m.__name__ != '__main__'
                 ]
    assert len(fn_modules) == 1
    return fn_modules[0]

In [9]:
%%fortran -v
subroutine read_params_exception(filename, params)
   implicit none
   ! Argument Declarations !
   character(len=*), intent(in) :: filename
   integer, dimension(4), intent(out) :: params

   ! (callback, hide) is most similar to an exception
   !f2py intent(callback, hide) raise_error
   external raise_error
   !f2py integer iostat
   !f2py character(len=256) iomsg
   !f2py call raise_error(iostat, iomsg)

   ! Variable Declarations
   integer :: i, iounit
   integer :: ios
   character(len=256) :: iom

   iounit = 1
   open(unit=iounit, status="old", file=filename, &
        err=10, iostat=ios, iomsg=iom)
   read(iounit, *, err=10, iostat=ios, iomsg=iom) (params(i), i=1, 4)
   close(iounit, err=20, iostat=ios, iomsg=iom)
   return
10 close(iounit)
20 call raise_error(ios, iom)

end subroutine read_params_exception


Ok. The following fortran objects are ready to use: read_params_exception


In [10]:
print(read_params_exception.__doc__)

params = read_params_exception(filename)

Wrapper for ``read_params_exception``.

Parameters
----------
filename : input string(len=-1)

Returns
-------
params : rank-1 array('i') with bounds (4)

Notes
-----
Call-back functions::

    def raise_error(iostat,iomsg): return 
    Required arguments:
        iostat : input int
        iomsg : input string(len=256)



In [11]:
def raise_error(ios, iom):
    raise IOError(ios, iom)

# For manual compile f2py module, simple:
#     import module
#     module.raise_error = ...
#     ...module.read_params_exception(...)
# But the function of compiling fortranmagic
# into a temporary hidden module
#
fmf2py_module('read_params_exception').raise_error = raise_error

read_params_exception(filenames[0])

array([1, 2, 3, 4], dtype=int32)

In [12]:
read_params_exception(filenames[1])

OSError: [Errno 5010] Bad integer for item 1 in list input                                                                                                                                                                                                                            

In [13]:
read_params_exception(filenames[2])

FileNotFoundError: [Errno 2] Cannot open file 'no-file': No such file or directory                                                                                                                                                                                                           